In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from scipy.io import loadmat 

import matplotlib.pyplot as plt
import seaborn as sns
sns.reset_orig()

In [2]:
import networkx as nx

from scipy.stats import bernoulli
from scipy.special import loggamma

from sklearn.preprocessing import OneHotEncoder

import tqdm

from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

In [3]:
import os
import glob
import pickle

In [4]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore', UserWarning)

In [5]:
blockmodels = importr("blockmodels")

In [6]:
base = importr("base")

In [7]:
dollar = base.__dict__["$"]

In [8]:
pd.options.display.max_rows = 1000

In [9]:
outdir = './output'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [10]:
from functools import lru_cache

@lru_cache(maxsize = 10000)
def normterm_discrete(n, k):
    if n == 1:
        return np.log(k)
    if k == 1:
        return 1.0
    elif k == 2:
        return np.sum(sorted([ np.exp(loggamma(n+1) - loggamma(t+1) - loggamma(n-t+1) + 
                               t*(np.log(t) - np.log(n)) + (n-t)*(np.log(n-t) - np.log(n))
                        )
                        for t in range(1, n)]))
    else:
        return normterm_discrete(n, k-1) + n/(k-2) * normterm_discrete(n, k-2)

In [11]:
def check_latent_index_variable(z):
    unique_z = sorted(np.unique(z))
    if len(unique_z) == np.max(z) + 1:
        return z
    new_z = np.zeros(z.shape, dtype=np.int)
    for index, current in enumerate(unique_z):
        new_z[z == current] = index
    return new_z

In [12]:
def calc_dnml(X, Z1, Z2, K=3, L=3):    
    N = X.shape[0]
    
    codelen_x_z = 0.0
    codelen_z = 0.0

    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg

            if n_all >=2:
                codelen_x_z += n_all * np.log(n_all)
                codelen_x_z += np.log(normterm_discrete(n_all, 2))
            
            if n_pos >=2:
                codelen_x_z -= n_pos * np.log(n_pos)
            if n_neg >=2:
                codelen_x_z -= n_neg * np.log(n_neg)
            
        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            codelen_z += n_k * (np.log(N) - np.log(n_k))

    codelen_z += np.log(normterm_discrete(N, K))
    
    codelen = codelen_x_z + codelen_z

    return codelen, codelen_x_z, codelen_z

In [13]:
def calc_dnml_by_prob(X, Z, alpha, theta, K=3, L=3):    
    N = X.shape[0]
    
    Z1 = np.argmax(Z, axis=1)
    
    Z1 = check_latent_index_variable(Z1)
    Z2 = Z1
    
    codelen_x_z = 0.0
    codelen_z = 0.0
        
    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg
            
            codelen_x_z += -n_pos * np.ma.log(theta[k, l]) - n_neg * np.ma.log(1.0 - theta[k, l])
            if n_all >=2:
                codelen_x_z += np.log(normterm_discrete(n_all, 2))
    
        n_k = np.sum(Z1 == k)
        codelen_z += -n_k * np.log(alpha[k])

    codelen_z += np.log(normterm_discrete(N, K))

    codelen = codelen_x_z + codelen_z

    return codelen, codelen_x_z, codelen_z

In [14]:
def calc_lsc(X, Z1, Z2, K=3, L=3):    
    codelen = 0.0
    N = X.shape[0]

    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg

            if n_all >=2:
                codelen += n_all * np.log(n_all)
            
            if n_pos >=2:
                codelen -= n_pos * np.log(n_pos)
            if n_neg >=2:
                codelen -= n_neg * np.log(n_neg)

        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            codelen += n_k * (np.log(N) - np.log(n_k))

        codelen += (k + (k+1)*(k+2))/2 * np.log(N/(2.0*np.pi)) -(k+1)/2 * np.log(2.0) + \
           (k+1) * loggamma((k+3)/2) - loggamma((k+1)*(k+3)/2) + (k+1)*(k+2)/2 * np.log(np.pi)

    return codelen

In [15]:
def estimate_sbm(X, K=10):
    numpy2ri.activate()
    sbm = blockmodels.BM_bernoulli(membership_type="SBM", adj=np.array(X),
                                       verbosity=0,
                                       exploration_factor=1.5,
                                       explore_min=K,
                                       explore_max=K)

    estimate = dollar(sbm, "estimate")
    estimate()

    theta_list = []
    pi_list = []
    z_posterior_list = []
    for k in range(K):
        n_clusters = k + 1
        theta = np.array(dollar(dollar(sbm, "model_parameters")[k], "pi"))
        z_posterior = np.array(dollar(dollar(sbm, "memberships")[k], "Z"))
        pi = np.sum(z_posterior, axis=0) + 10 * EPS
        pi /= np.sum(pi)

        theta_list.append(theta)
        z_posterior_list.append(z_posterior)
        pi_list.append(pi)
        
    numpy2ri.deactivate()

    return pi_list, theta_list, z_posterior_list

In [16]:
files_pairs = sorted(glob.glob('./data/enron/*.pairs'))

In [17]:
with open('addresses.txt', 'r') as f:
    addresses = np.array(f.read().splitlines())

In [18]:
n_term = len(files_pairs)
n_employee = len(addresses)
X = np.zeros((n_term, n_employee, n_employee), dtype=np.int)

for k, file in enumerate(files_pairs):
    print(file)
    try:
        sender_receiver = np.loadtxt(file, delimiter='\t', dtype=np.int)
        if len(sender_receiver.shape) == 1:
            sender_receiver = sender_receiver.reshape(1, -1)
        id_sender = sender_receiver[:, 0] - 1
        id_receiver = sender_receiver[:, 1] - 1
        X[k, id_sender, id_receiver] += 1
    except:
        continue

./data/enron/19990510.pairs
./data/enron/19990517.pairs
./data/enron/19990524.pairs
./data/enron/19990531.pairs
./data/enron/19990607.pairs
./data/enron/19990614.pairs
./data/enron/19990621.pairs
./data/enron/19990628.pairs
./data/enron/19990705.pairs
./data/enron/19990712.pairs
./data/enron/19990719.pairs
./data/enron/19990726.pairs
./data/enron/19990802.pairs
./data/enron/19990809.pairs
./data/enron/19990816.pairs
./data/enron/19990823.pairs
./data/enron/19990830.pairs
./data/enron/19990906.pairs
./data/enron/19990913.pairs
./data/enron/19990920.pairs
./data/enron/19990927.pairs
./data/enron/19991004.pairs
./data/enron/19991011.pairs
./data/enron/19991018.pairs
./data/enron/19991025.pairs
./data/enron/19991101.pairs
./data/enron/19991108.pairs
./data/enron/19991115.pairs
./data/enron/19991122.pairs
./data/enron/19991129.pairs
./data/enron/19991206.pairs
./data/enron/19991213.pairs
./data/enron/19991220.pairs
./data/enron/19991227.pairs
./data/enron/20000103.pairs
./data/enron/2000011

In [19]:
T = np.where(np.sum(X, axis=(1, 2)) == 0)[0][0]

In [20]:
X = X[:T, :, :]

In [21]:
with open('n_emails_w157.pkl', 'wb') as f:
    pickle.dump(X, f)

In [ ]:
EPS = np.finfo(np.float).eps

T = X.shape[0]

K = 10

numpy2ri.activate()    

for h in [1, 2, 4 ,8, 16]:
    pi_all, theta_all, z_all = [], [], []
    pi_f, theta_f, z_f = [], [], []
    pi_l, theta_l, z_l = [], [], []

    for t in tqdm.tqdm(range(h, T-h)):
        X_all_t = (np.sum(X[(t-h):(t+h), :, :], axis=0) > 0).astype(np.int)
        X_f_t = (np.sum(X[(t-h):t, :, :], axis=0) > 0).astype(np.int)
        X_l_t = (np.sum(X[t:(t+h), :, :], axis=0) > 0).astype(np.int)
        
        pi_all_t, theta_all_t, z_all_t = estimate_sbm(X_all_t)
        pi_all.append(pi_all_t)
        theta_all.append(theta_all_t)
        z_all.append(z_all_t)

        pi_f_t, theta_f_t, z_f_t = estimate_sbm(X_f_t)
        pi_f.append(pi_f_t)
        theta_f.append(theta_f_t)
        z_f.append(z_f_t)
        
        pi_l_t, theta_l_t, z_l_t = estimate_sbm(X_l_t)
        pi_l.append(pi_l_t)
        theta_l.append(theta_l_t)
        z_l.append(z_l_t)

    with open(os.path.join(outdir, 'pi_enron_all_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(pi_all, f)
    with open(os.path.join(outdir, 'theta_enron_all_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(theta_all, f)
    with open(os.path.join(outdir, 'z_enron_all_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(z_all, f)
        
    with open(os.path.join(outdir, 'pi_enron_f_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(pi_f, f)
    with open(os.path.join(outdir, 'theta_enron_f_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(theta_f, f)
    with open(os.path.join(outdir, 'z_enron_f_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(z_f, f)

    with open(os.path.join(outdir, 'pi_enron_l_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(pi_l, f)
    with open(os.path.join(outdir, 'theta_enron_l_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(theta_l, f)
    with open(os.path.join(outdir, 'z_enron_l_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(z_l, f)

  0%|          | 0/155 [00:00<?, ?it/s]

  1%|          | 1/155 [00:23<1:01:15, 23.87s/it]

  1%|▏         | 2/155 [00:37<52:40, 20.66s/it]  

  2%|▏         | 3/155 [00:57<52:23, 20.68s/it]

  3%|▎         | 4/155 [01:16<50:42, 20.15s/it]

...

 52%|█████▏    | 65/125 [20:56<17:22, 17.38s/it]

 53%|█████▎    | 66/125 [21:16<17:43, 18.02s/it]

 54%|█████▎    | 67/125 [21:31<16:32, 17.11s/it]

 54%|█████▍    | 68/125 [21:47<16:01, 16.88s/it]

 55%|█████▌    | 69/125 [22:04<15:43, 16.84s/it]